In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [25]:
def delta_days_and_current(tickers, dates, delta=7):
    """This function obtains, for each pair of ticker and date, the closing price of the ticker delta days
    after the given date and the closing price of the ticker for the day of the reference date.
    
    For the inputs:
    tickers: List of tickers, each represented by a string. Same length as dates!
    dates: List of dates, each represented in the format %Y-%m-%d (e.g. 2010-01-24)
    delta: Number of days after the reference date from which to obtain the previous price. It can also be a list,
        with as many deltas as desired.
    
    The output is a pandas dataframe, with as many rows as specified tickers, and columns Reference Date, 
    Previous Close, and Current Close."""
    

    if type(delta) == int:
        delta = [delta]
    
    results = {field: [] for field in 
               ['Ticker', "Reference Date", "Current Close"] + \
               [f"Close_Price_{abs(d)}_Days_Before" for d in delta if d < 0] + \
               [f"Close_Price_{d}_Days_Later" for d in delta if d > 0]}
    
    #This unelegant move is because im lazy
    delta = [-d for d in delta]
        
    for i, t in enumerate(tickers):
        #If date falls in weekends, take Friday and Monday
        extra_add = 0
        if datetime.strptime(dates[i], '%Y-%m-%d').isoweekday() == 6:
            extra_add = -1
        elif datetime.strptime(dates[i], '%Y-%m-%d').isoweekday() == 7:
            extra_add = 1
        
        current = datetime.strptime(dates[i], '%Y-%m-%d') + timedelta(days=extra_add)
        
        if max(delta) >= 0:
            max_previous = current + timedelta(days=-max(delta))
            if min(delta) > 0:
                max_next = current
            else:
                max_next = current + timedelta(days=-min(delta))    
        else:
            max_next = current + timedelta(days=-min(delta)) 
            max_previous = current
            
        data = yf.download(t, start=max_previous + timedelta(days=-2), end=max_next + timedelta(days=2))
        current_close = data.loc[data.index == current, 'Close'][0]
        #print(data[['Close']])
        results['Ticker'].append(t)
        results["Reference Date"].append(current)
        results["Current Close"].append(current_close)
        
        for d in delta:
            if d != 0:
                previous = current + timedelta(days=-d)

                #If date falls in weekends, take Friday and Monday
                if previous.isoweekday() == 6:
                    previous += timedelta(days=-1)
                elif previous.isoweekday() == 7:
                    previous += timedelta(days=+1)

                previous_close = data.loc[data.index == previous, 'Close'][0]

                if d > 0:
                    results[f"Close_Price_{d}_Days_Before"].append(previous_close)
                elif d < 0:
                    results[f"Close_Price_{abs(d)}_Days_Later"].append(previous_close)

    results = pd.DataFrame(results).set_index('Ticker')
    return results

In [26]:
tickers = ['AAPL', 'GOOG']
dates = ['2012-03-15', '2010-08-21']
deltas = [7, 5, 3, -1, -2, -5]
df = delta_days_and_current(tickers, dates, deltas)
df

[*********************100%***********************]  1 of 1 completed
                Close
Date                 
2012-03-07  18.953215
2012-03-08  19.356787
2012-03-09  19.470358
2012-03-12  19.714287
2012-03-13  20.289286
2012-03-14  21.056429
2012-03-15  20.912857
2012-03-16  20.913214
2012-03-19  21.467857
2012-03-20  21.641430
2012-03-21  21.517857
2012-03-22  21.405001
2012-03-23  21.287500
[*********************100%***********************]  1 of 1 completed
                 Close
Date                  
2010-08-12  245.086349
2010-08-13  242.266922
2010-08-16  241.888336
2010-08-17  244.344131
2010-08-18  240.174759
2010-08-19  233.111237
2010-08-20  230.147354
2010-08-23  231.168518
2010-08-24  224.852203
2010-08-25  226.461166
2010-08-26  224.647964
2010-08-27  228.558304


Reference Date  Current Close  Close_Price_1_Days_Before  \
Ticker                                                            
AAPL       2012-03-15      20.912857                  21.056429   
GOOG       2010-08-20     230.147354                 233.111237   

        Close_Price_2_Days_Before  Close_Price_5_Days_Before  \
Ticker                                                         
AAPL                    20.289286                  19.470358   
GOOG                   240.174759                 241.888336   

        Close_Price_7_Days_Later  Close_Price_5_Days_Later  \
Ticker                                                       
AAPL                   21.405001                 21.641430   
GOOG                  228.558304                226.461166   

        Close_Price_3_Days_Later  
Ticker                            
AAPL                   21.467857  
GOOG                  231.168518